# Pairwise RMSD calculation

## Import necessary libraries

In [1]:
# Import necessary modules
from pymol import cmd
from rms_calculator import calculate_rms_stats as calc_rms
import ipywidgets as widgets
from IPython.display import display
import tempfile
import os


## Import structure

Input a pdb code or a local file, then click on "Load Structure".

You may use **1A1T** as an example pdb code.

In [2]:
from pymol import cmd
import ipywidgets as widgets
from IPython.display import display
import tempfile
import os

# Widget inputs
pdb_id_input = widgets.Text(description='PDB ID:')
file_upload = widgets.FileUpload(accept='.pdb,.cif', multiple=False)
load_button = widgets.Button(description="Load Structure")
display(widgets.Label("Enter PDB ID or upload a file:"), pdb_id_input, file_upload, load_button)

# Variable to store current object name
current_structure = None

def load_structure(b):
    global current_structure  # allow modification of outer variable

    pdb_id = pdb_id_input.value.strip()
    if pdb_id:
        try:
            cmd.fetch(pdb_id)
            current_structure = pdb_id  # object name is PDB ID by default
            print(f"Fetched {pdb_id} from PDB.")
        except Exception as e:
            print(f"Error fetching PDB ID {pdb_id}: {e}")
    elif file_upload.value:
        for name, file_info in file_upload.value.items():
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(name)[1]) as tmp:
                tmp.write(file_info['content'])
                tmp_path = tmp.name
            try:
                # Use the filename (without extension) as the PyMOL object name
                obj_name = os.path.splitext(name)[0]
                cmd.load(tmp_path, obj_name)
                current_structure = obj_name
                print(f"Loaded {name} into PyMOL as {obj_name}.")
            except Exception as e:
                print(f"Error loading file {name}: {e}")
    else:
        print("Please enter a PDB ID or upload a file.")

load_button.on_click(load_structure)


Label(value='Enter PDB ID or upload a file:')

Text(value='', description='PDB ID:')

FileUpload(value=(), accept='.pdb,.cif', description='Upload')

Button(description='Load Structure', style=ButtonStyle())

In absence of user input, structure 1A1T will be loaded by default.

In [3]:
# Fallback default after widget cell
if current_structure is None:
    default_pdb = "1A1T"
    try:
        cmd.fetch(default_pdb)
        current_structure = default_pdb
        print(f"No structure loaded via widget. Defaulted to {default_pdb}.")
    except Exception as e:
        print(f"Error fetching default PDB {default_pdb}: {e}")


No structure loaded via widget. Defaulted to 1A1T.


## Calculate pairwise RMSD

### All atoms

In [4]:
full_structure = calc_rms(
    current_structure,
    selection="all",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############
Note: no explicit selection was provided, so the script selected all atoms in the object.
If you want a subset (for example only chain A), pass a selection string via the 'selection' argument, e.g.: selection='chain A'
It is not necessary to explicitely exlude hydrogen atoms. RMS statistics are calculated both including and excluding hydrogens automatically.


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (all)
States inspected: 25
Atoms selected:
  All: 1516
  Heavy (no H): 873
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
---------------------------------------------------

### Selected atoms

Below are some selections that are relevant to 1A1T (mix of protein, RNA and
Zinc ions). You may modify the selections as needed using the [pymol selection algebra](https://pymolwiki.org/index.php/Selection_Algebra).

#### All backbone atoms (proteins and nucleic acids)

In [5]:
backbone = calc_rms(
    current_structure,
    selection="backbone",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (backbone)
States inspected: 25
Atoms selected:
  All: 654
  Heavy (no H): 460
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
------------------------------------------------------------
All atoms            654       600      1.966      0.590
Heavy atoms          460       600      2.017      0.635
------------------------------------------------------------
Average RMS per state (showing NaN where unavailable):

All atoms:
State     Mean RMS     #pairs
----------------------------------
    1        2.085         24
    2      

#### Protein backbone atoms

In [6]:
prot_backbone = calc_rms(
    current_structure,
    selection="chain A and backbone",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (chain A and backbone)
States inspected: 25
Atoms selected:
  All: 274
  Heavy (no H): 220
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
------------------------------------------------------------
All atoms            274       600      2.031      0.822
Heavy atoms          220       600      2.050      0.854
------------------------------------------------------------
Average RMS per state (showing NaN where unavailable):

All atoms:
State     Mean RMS     #pairs
----------------------------------
    1        2.371         24

#### RNA backbone atoms

In [7]:
RNA_backbone = calc_rms(
    current_structure,
    selection="chain B and backbone",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (chain B and backbone)
States inspected: 25
Atoms selected:
  All: 380
  Heavy (no H): 240
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
------------------------------------------------------------
All atoms            380       600      1.295      0.328
Heavy atoms          240       600      1.287      0.343
------------------------------------------------------------
Average RMS per state (showing NaN where unavailable):

All atoms:
State     Mean RMS     #pairs
----------------------------------
    1        1.250         24

#### All atoms excluding zinc ions

In [8]:
no_zn = calc_rms(
    current_structure,
    selection="not resn ZN",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (not resn ZN)
States inspected: 25
Atoms selected:
  All: 1514
  Heavy (no H): 871
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
------------------------------------------------------------
All atoms           1514       600      2.557      0.576
Heavy atoms          871       600      2.320      0.597
------------------------------------------------------------
Average RMS per state (showing NaN where unavailable):

All atoms:
State     Mean RMS     #pairs
----------------------------------
    1        2.784         24
    2  

#### Zinc binding site atoms

Here we define zinc binding site atoms as those within 10 Å of any zinc ion.

In [9]:
zn_binding_site = calc_rms(
    current_structure,
    selection="resn ZN around 10",
    quiet=False,
    display_level=1,
    export_report=False,
    report_path="."
)



*****************************
* RMS Statistics Calculator *
*****************************

The level of detail in this report has been set to => 1 (global average and per-state means included)

############ Data selection ############

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Resolved selection: (1A1T) and (resn ZN around 10)
States inspected: 25
Atoms selected:
  All: 693
  Heavy (no H): 365
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

########################################

======================== RMS Summary ========================
Mode              #atoms   #values       mean         sd
------------------------------------------------------------
All atoms            693       600      2.714      0.825
Heavy atoms          365       600      2.489      0.897
------------------------------------------------------------
Average RMS per state (showing NaN where unavailable):

All atoms:
State     Mean RMS     #pairs
----------------------------------
    1        3.293         24
  

## Formatted output

Example of formatted results:

In [10]:
## Full structure overall mean RMS (no hydrogens)
mean_no_hydrogen = full_structure["modes"]["no_hydrogen"]["overall_mean"]
sd_no_hydrogen = full_structure["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (no hydrogens) for {current_structure}: {mean_no_hydrogen:.2f} +/- {sd_no_hydrogen:.2f} Å, calculated over {full_structure['states']} states.")

mean_backbone_no_hydrogen = backbone["modes"]["no_hydrogen"]["overall_mean"]
sd_backbone_no_hydrogen = backbone["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (backbone, no hydrogens) for {current_structure}: {mean_backbone_no_hydrogen:.2f} +/- {sd_backbone_no_hydrogen:.2f} Å, calculated over {backbone['states']} states.")

mean_prot_backbone_no_hydrogen = prot_backbone["modes"]["no_hydrogen"]["overall_mean"]
sd_prot_backbone_no_hydrogen = prot_backbone["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (protein backbone, no hydrogens) for {current_structure}: {mean_prot_backbone_no_hydrogen:.2f} +/- {sd_prot_backbone_no_hydrogen:.2f} Å, calculated over {prot_backbone['states']} states.")

mean_RNA_backbone_no_hydrogen = RNA_backbone["modes"]["no_hydrogen"]["overall_mean"]
sd_RNA_backbone_no_hydrogen = RNA_backbone["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (RNA backbone, no hydrogens) for {current_structure}: {mean_RNA_backbone_no_hydrogen:.2f} +/- {sd_RNA_backbone_no_hydrogen:.2f} Å, calculated over {RNA_backbone['states']} states.")

mean_no_zn_no_hydrogen = no_zn["modes"]["no_hydrogen"]["overall_mean"]
sd_no_zn_no_hydrogen = no_zn["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (no ZN, no hydrogens) for {current_structure}: {mean_no_zn_no_hydrogen:.2f} +/- {sd_no_zn_no_hydrogen:.2f} Å, calculated over {no_zn['states']} states.")

mean_zn_binding_site_no_hydrogen = zn_binding_site["modes"]["no_hydrogen"]["overall_mean"]
sd_zn_binding_site_no_hydrogen = zn_binding_site["modes"]["no_hydrogen"]["overall_sd"]
print(f"Overall mean RMS (ZN binding site, no hydrogens) for {current_structure}: {mean_zn_binding_site_no_hydrogen:.2f} +/- {sd_zn_binding_site_no_hydrogen:.2f} Å, calculated over {zn_binding_site['states']} states.")


Overall mean RMS (no hydrogens) for 1A1T: 2.32 +/- 0.60 Å, calculated over 25 states.
Overall mean RMS (backbone, no hydrogens) for 1A1T: 2.02 +/- 0.63 Å, calculated over 25 states.
Overall mean RMS (protein backbone, no hydrogens) for 1A1T: 2.05 +/- 0.85 Å, calculated over 25 states.
Overall mean RMS (RNA backbone, no hydrogens) for 1A1T: 1.29 +/- 0.34 Å, calculated over 25 states.
Overall mean RMS (no ZN, no hydrogens) for 1A1T: 2.32 +/- 0.60 Å, calculated over 25 states.
Overall mean RMS (ZN binding site, no hydrogens) for 1A1T: 2.49 +/- 0.90 Å, calculated over 25 states.
